In [80]:
import psycopg2
from sqlalchemy import create_engine
from configparser import ConfigParser
from pathlib import Path
import os
import io
import pandas as pd

In [26]:
config = ConfigParser()
file = '../Notebooks/config.ini'
config.read(file)

USER_BASE = config['database']['user']
PASSWORD_BASE = config['database']['passw']
DATABASE = config['database']['db']
PORT = config['database']['port']
HOST = config['database']['host']

In [72]:
def conn():
    connetion = psycopg2.connect(host=HOST, dbname=DATABASE, user=USER_BASE, password=PASSWORD_BASE)
    conn = connetion.raw_connection()
    return conn
    

In [85]:
datos = pd.read_csv('D:\\emissionsmap\\SubirData\\Normalizados\\Tabla_Hecho.csv')

In [86]:
datos.head()

,Año,País,Combustible,Continente,Id_Año,Id_País,Id_Combustible,Id_Continente
0,1965,World,Biomass,País,d46e1fcf4c07ce4a69ee07e4134bcef1,f5a7924e621e84c9280a9a27e1bcb7f6,57a34c10edc9db4cc4fbfc06367285f8,f0aa1982f61314b9aab258c2f110ebab
1,1966,Afghanistan,Coal,Código,3683af9d6f6c06acee72992f2977f67e,6e9767f4f15357401b148b9a8ced14ef,ff8cfc4e6a79e6514be2533a5f5506ad,c54e67f2ff1683617c1edadad20f99bb
2,1967,Albania,Cogeneration,Año,a82d922b133be19c1171534e6594f754,c420ddff824a5c0eec70dd23d62496bc,be8c640ab7313b88c232af9328ebf5a8,c55baca5e562cecdd18fb070c0ff8f80
3,1968,Algeria,Gas,Combustibles Fósiles (% Energía Primaria Equiv...,98c7242894844ecd6ec94af67ac8247d,575b9408b6daa2ddcefbcf6d81c9b4c9,635459702f9817de29e608fa3395637c,50a5739c47db56022e3798a1f85637cd
4,1969,American Samoa,Geothermal,Renovables (% Energía Primaria Equivalente),4d8556695c262ab91ff51a943fdd6058,b86f99753a08d00fa3ca946d07ecac29,3f3f7ddaa26a792f9707f4de5f077cf3,af1b6e0ed591be5893f58516a3f64119


In [91]:
def cargarTablas(data, nombre):
    create_connection = f'postgresql+psycopg2://{USER_BASE}:{PASSWORD_BASE}@{HOST}:{PORT}/{DATABASE}'
    engine = create_engine(create_connection)

    data.head(0).to_sql(nombre, engine, if_exists='replace',index=False) #drops old table and creates new empty table

    conn = engine.raw_connection()
    cur = conn.cursor()
    output = io.StringIO()
    data.to_csv(output, sep='\t', header=False, index=False)
    output.seek(0)
    contents = output.getvalue()
    cur.copy_from(output, nombre, null='')
    
    return('Tabala cargadas Exitosamente')

In [100]:
def consultas(consulta):
    try:
        connetion = psycopg2.connect(host=HOST, dbname=DATABASE, user=USER_BASE, password=PASSWORD_BASE)
        micursor = connetion.cursor()
        micursor.execute(consulta)
        df = micursor.fetchall()
        df = list(df)
        df = pd.DataFrame(df)
                    
    except Exception as error:
        print(error)
    finally:
        if micursor is not None:
            micursor.close()
        if connetion is not None:
            connetion.close()
        return df

In [92]:
cargarTablas(datos, 'prueba')

'Tabala cargadas Exitosamente'

In [107]:
df = consultas('SELECT * FROM gases') #consulta de tablas


In [103]:
dd = consultas('select table_schema,table_name,column_name,data_type from information_schema.columns order by column_name') #lista toda las tablas y campos

In [106]:
dd[dd[1] == 'gases']

,0,1,2,3
98,public,gases,Año,bigint
224,public,gases,Id_Año,text
253,public,gases,Medios Canal 4,double precision
254,public,gases,Medios De Co2,double precision
255,public,gases,Medios N20,double precision
256,public,gases,Medios Sf6,double precision
322,public,gases,Unc Ch4,double precision
323,public,gases,Unc Co2,double precision
324,public,gases,Unc N2O,double precision
325,public,gases,Unc Sf6,double precision
